In [1]:
!pip install ultralytics torch torchvision pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.1 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.3 MB/s eta 0:00:00:00:0100:01
  Attempting unins

In [2]:
import os
import yaml
import shutil
from ultralytics import YOLO

def create_dataset_yaml(working_dir, data_dir, class_names):
    """
    Создает YAML-файл конфигурации датасета в рабочей директории
    """
    yaml_content = {
        'path': data_dir,  # путь к данным
        'train': 'images/train',
        'val': 'images/val',
        'names': class_names
    }
    
    yaml_path = os.path.join(working_dir, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    
    return yaml_path

def verify_dataset(data_dir):
    """
    Проверяет соответствие изображений и разметок
    """
    for split in ['train', 'val']:
        images_dir = os.path.join(data_dir, 'images', split)
        labels_dir = os.path.join(data_dir, 'labels', split)
        
        if not os.path.exists(images_dir):
            raise ValueError(f"Директория {images_dir} не существует")
        if not os.path.exists(labels_dir):
            raise ValueError(f"Директория {labels_dir} не существует")
        
        # Проверяем соответствие файлов
        images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        labels = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
        
        image_names = {os.path.splitext(f)[0] for f in images}
        label_names = {os.path.splitext(f)[0] for f in labels}
        
        if image_names != label_names:
            missing_images = label_names - image_names
            missing_labels = image_names - label_names
            
            if missing_images:
                print(f"Предупреждение: Отсутствуют изображения для {len(missing_images)} разметок")
            if missing_labels:
                print(f"Предупреждение: Отсутствуют разметки для {len(missing_labels)} изображений")

def train_yolov8(data_dir, model_size='n', epochs=100, imgsz=640):
    """
    Обучение модели YOLOv8
    """
    working_dir = '/kaggle/working/'
    
    class_names = ['dark_broken', 'dark_entire', 'red_broken', 'red_entire']
    
    # Проверяем датасет
    print("Проверка датасета...")
    verify_dataset(data_dir)
    
    # Создаем конфигурационный файл в рабочей директории
    print("Создание конфигурации датасета...")
    yaml_path = create_dataset_yaml(working_dir, data_dir, class_names)
    
    # Загружаем модель
    print(f"Загрузка модели YOLOv8{model_size}...")
    model = YOLO(f'yolov8{model_size}.pt')
    
    # Обучаем модель
    print("Запуск обучения...")
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        lr0=0.01,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        patience=0,
        save=True,
        save_period=-1,
        cache=False, 
        workers=2,   
        single_cls=False,
        optimizer='SGD',
        val=True,
        device=0 if torch.cuda.is_available() else None, 
        verbose=True,
        project=working_dir,
        name='yolov8_train',
        exist_ok=True
    )
    
    print("Обучение завершено!")
    
    # Копируем лучшую модель в видимую локацию
    best_model_path = os.path.join(working_dir, 'yolov8_train/weights/best.pt')
    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, os.path.join(working_dir, 'best_model.pt'))
        print(f"Лучшая модель сохранена как: {os.path.join(working_dir, 'best_model.pt')}")
    
    return results

# Для Kaggle Notebook
if __name__ == "__main__":
    import torch
    
    # Конфигурация
    DATA_DIR = "/kaggle/input/mlbdlw4"  
    MODEL_SIZE = 'n'  # n, s, m, l, x
    EPOCHS = 300  
    IMGSZ = 640
    
    # Проверяем доступность GPU
    print(f"Доступен GPU: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # Запуск обучения
    results = train_yolov8(
        data_dir=DATA_DIR,
        model_size=MODEL_SIZE,
        epochs=EPOCHS,
        imgsz=IMGSZ
    )

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Доступен GPU: True
GPU: Tesla T4
Проверка датасета...
Создание конфигурации датасета...
Загрузка модели YOLOv8n...
Запуск обучения...
Ultralytics 8.3.204 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, fo

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         81         60      0.714      0.562      0.643      0.266
           dark_broken          4          4      0.344        0.5      0.497      0.327
           dark_entire         19         19      0.808      0.667      0.767      0.242
            red_broken          9          9      0.991      0.222      0.406      0.113
            red_entire         28         28      0.711      0.857      0.899       0.38
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /kaggle/working/yolov8_train
Обучение завершено!
Лучшая модель сохранена как: /kaggle/working/best_model.pt


In [24]:
import cv2
import torch
from ultralytics import YOLO
import time

def analyze_every_frame(video_path, model_path, output_path=None):
    # Загружаем модель
    print("Загрузка модели...")
    model = YOLO(model_path)
    
    # Открываем видео
    print(f"Открываем видео: {video_path}")
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Ошибка: не удалось открыть видеофайл")
        return
    
    # Получаем параметры видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0
    
    print(f"Параметры видео:")
    print(f"  FPS: {fps:.2f}")
    print(f"  Всего кадров: {total_frames}")
    print(f"  Длительность: {duration:.2f} сек")
    print(f"  Режим: анализ каждого кадра")
    
    # Подготовка для сохранения результата
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        out_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out = cv2.VideoWriter(output_path, fourcc, fps, (out_width, out_height))
    
    # Словарь для классов
    class_names = ['dark_broken', 'dark_entire', 'red_broken', 'red_entire']
    
    # Переменные для анализа
    current_frame = 0
    total_detections = 0
    
    print("\nНачинаем анализ каждого кадра видео...")
    print("-" * 60)
    
    # Для измерения производительности
    start_time = time.time()
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Получаем время в видео
        current_time = current_frame / fps
        
        # Выполняем предсказание для текущего кадра
        results = model(frame, verbose=False)
        
        # Анализируем результаты
        frame_detections = 0
        if len(results) > 0 and results[0].boxes is not None:
            boxes = results[0].boxes
            
            if len(boxes) > 0:
                # Обрабатываем все обнаруженные объекты
                for i, box in enumerate(boxes):
                    cls_id = int(box.cls[0])
                    conf = float(box.conf[0])
                    class_name = class_names[cls_id]
                    
                    # Выводим информацию в консоль
                    print(f"Кадр: {current_frame:5d} | "
                          f"Время: {current_time:6.2f} сек | "
                          f"Мишень: {class_name:12} | "
                          f"Уверенность: {conf:.3f}")
                    
                    frame_detections += 1
                    total_detections += 1
                    
                    # Рисуем bounding box на frame (если нужно сохранить видео)
                    if output_path:
                        xyxy = box.xyxy[0].cpu().numpy()
                        x1, y1, x2, y2 = map(int, xyxy)
                        
                        # Цвет в зависимости от класса
                        if 'red' in class_name:
                            color = (0, 0, 255)  # Красный для красных мишеней
                        else:
                            color = (0, 0, 0)    # Черный для темных мишеней
                        
                        # Рисуем прямоугольник
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        
                        # Подпись с классом и уверенностью
                        label = f"{class_name} {conf:.2f}"
                        cv2.putText(frame, label, (x1, y1-10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Если в кадре нет обнаружений
        if frame_detections == 0:
            print(f"Кадр: {current_frame:5d} | "
                  f"Время: {current_time:6.2f} сек | "
                  f"Мишеней не обнаружено")
        
        # Сохраняем кадр в выходное видео (если нужно)
        if output_path:
            out.write(frame)
        
        current_frame += 1
        
        # Прогресс каждые 100 кадров или каждую секунду
        if current_frame % 100 == 0 or current_frame % int(fps) == 0:
            elapsed_time = time.time() - start_time
            frames_per_second = current_frame / elapsed_time if elapsed_time > 0 else 0
            progress = (current_frame / total_frames) * 100
            print(f"Прогресс: {progress:.1f}% | "
                  f"Обработано: {current_frame}/{total_frames} | "
                  f"Скорость: {frames_per_second:.1f} кадр/сек")
    
    # Завершение
    elapsed_time = time.time() - start_time
    print("-" * 60)
    print(f"Анализ завершен!")
    print(f"Обработано кадров: {current_frame}")
    print(f"Обнаружено мишеней: {total_detections}")
    print(f"Общее время обработки: {elapsed_time:.2f} сек")
    print(f"Средняя скорость: {current_frame/elapsed_time:.2f} кадр/сек")
    
    # Освобождаем ресурсы
    cap.release()
    if output_path:
        out.release()
        print(f"Видео с разметкой сохранено как: {output_path}")


if __name__ == "__main__":
    # Конфигурация
    VIDEO_PATH = "/kaggle/input/test-video-2-mp4/SHOT0048.MP4"
    MODEL_PATH = "/kaggle/working/best_model.pt" 
    OUTPUT_VIDEO_PATH = "/kaggle/working/analyzed_video.mp4"
    
    # Проверяем доступность GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используемое устройство: {device}")
    
    # Запускаем анализ
    analyze_every_frame(
        video_path=VIDEO_PATH,
        model_path=MODEL_PATH,
        output_path=OUTPUT_VIDEO_PATH
    )


Используемое устройство: cuda
Загрузка модели...
Открываем видео: /kaggle/input/test-video-2-mp4/SHOT0048.MP4
Параметры видео:
  FPS: 39.96
  Всего кадров: 464
  Длительность: 11.61 сек
  Режим: анализ каждого кадра

Начинаем анализ каждого кадра видео...
------------------------------------------------------------
Кадр:     0 | Время:   0.00 сек | Мишеней не обнаружено
Кадр:     1 | Время:   0.03 сек | Мишеней не обнаружено
Кадр:     2 | Время:   0.05 сек | Мишеней не обнаружено
Кадр:     3 | Время:   0.08 сек | Мишеней не обнаружено
Кадр:     4 | Время:   0.10 сек | Мишеней не обнаружено
Кадр:     5 | Время:   0.13 сек | Мишеней не обнаружено
Кадр:     6 | Время:   0.15 сек | Мишеней не обнаружено
Кадр:     7 | Время:   0.18 сек | Мишеней не обнаружено
Кадр:     8 | Время:   0.20 сек | Мишеней не обнаружено
Кадр:     9 | Время:   0.23 сек | Мишеней не обнаружено
Кадр:    10 | Время:   0.25 сек | Мишеней не обнаружено
Кадр:    11 | Время:   0.28 сек | Мишеней не обнаружено
Кадр:    12

In [11]:
import os
import cv2
import torch
import pandas as pd
from ultralytics import YOLO
import time
from pathlib import Path

def analyze_videos_in_directory(video_dir, model_path, output_csv):
    """
    Анализирует все видео в директории и сохраняет результаты в CSV

    """
    
    # Загружаем модель
    print("Загрузка модели...")
    model = YOLO(model_path)
    
    # Словарь для классов
    class_names = ['dark_broken', 'dark_entire', 'red_broken', 'red_entire']
    
    # Список для хранения результатов
    results = []
    
    # Поддерживаемые форматы видео
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv', '.webm']
    
    # Находим все видеофайлы в директории
    video_files = []
    for ext in video_extensions:
        video_files.extend(Path(video_dir).glob(f'*{ext}'))
        video_files.extend(Path(video_dir).glob(f'*{ext.upper()}'))
    
    print(f"Найдено видеофайлов: {len(video_files)}")
    
    if not video_files:
        print("Видеофайлы не найдены!")
        return
    
    # Обрабатываем каждое видео
    for video_path in video_files:
        print(f"\nОбработка видео: {video_path.name}")
        
        # Открываем видео
        cap = cv2.VideoCapture(str(video_path))
        
        if not cap.isOpened():
            print(f"Ошибка: не удалось открыть видео {video_path.name}")
            continue
        
        # Получаем параметры видео
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        print(f"  FPS: {fps:.2f}, Кадров: {total_frames}, Разрешение: {width}x{height}")
        
        # Переменные для отслеживания прогресса
        frame_count = 0
        start_time = time.time()
        
        while True:
            ret, frame = cap.read()
            
            if not ret:
                break
            
            # Получаем время в видео
            current_time = frame_count / fps
            
            # Выполняем предсказание
            prediction_results = model(frame, verbose=False, conf=0.3)
            
            # Анализируем результаты
            if len(prediction_results) > 0 and prediction_results[0].boxes is not None:
                boxes = prediction_results[0].boxes
                
                if len(boxes) > 0:
                    # Обрабатываем все обнаруженные объекты
                    for i, box in enumerate(boxes):
                        cls_id = int(box.cls[0])
                        conf = float(box.conf[0])
                        class_name = class_names[cls_id]
                        
                        # Разделяем на цвет и состояние
                        color, state = class_name.split('_')
                        
                        # Получаем координаты bounding box
                        xyxy = box.xyxy[0].cpu().numpy()
                        x1, y1, x2, y2 = map(int, xyxy)
                        
                        # Вычисляем центр мишени
                        center_x = (x1 + x2) / 2
                        center_y = (y1 + y2) / 2
                        
                        # Определяем положение мишени относительно кадра
                        position_x = "left" if center_x < width/3 else "center" if center_x < 2*width/3 else "right"
                        position_y = "top" if center_y < height/3 else "middle" if center_y < 2*height/3 else "bottom"
                        position = f"{position_y}-{position_x}"
                        
                        # Добавляем результат в список
                        results.append({
                            'video_file': video_path.name,
                            'target_type': color,  # 'dark' или 'red'
                            'target_state': state,  # 'broken' или 'entire'
                            'target_position': position,
                            'frame_time_sec': round(current_time, 2),
                            'confidence': round(conf, 3),
                            'bbox_center_x': int(center_x),
                            'bbox_center_y': int(center_y),
                            'frame_number': frame_count
                        })
            
            frame_count += 1
            
            # Показываем прогресс каждые 5 секунд видео
            if frame_count % int(fps * 5) == 0:
                progress = (frame_count / total_frames) * 100
                elapsed = time.time() - start_time
                fps_actual = frame_count / elapsed if elapsed > 0 else 0
                print(f"  Прогресс: {progress:.1f}% | Скорость: {fps_actual:.1f} кадр/сек")
        
        # Закрываем видео
        cap.release()
        
        # Статистика по видео
        video_results = [r for r in results if r['video_file'] == video_path.name]
        print(f"  Обработано кадров: {frame_count}")
        print(f"  Обнаружено мишеней: {len(video_results)}")
        
        if video_results:
            # Анализируем типы обнаруженных мишеней
            target_types = {}
            for r in video_results:
                key = f"{r['target_type']}_{r['target_state']}"
                target_types[key] = target_types.get(key, 0) + 1
            
            print(f"  Распределение мишеней: {target_types}")
    
    # Сохраняем результаты в CSV
    if results:
        df = pd.DataFrame(results)
        
        # Сортируем по имени файла и времени
        df = df.sort_values(['video_file', 'frame_time_sec'])
        
        # Сохраняем в CSV
        df.to_csv(output_csv, index=False)
        print(f"\nРезультаты сохранены в: {output_csv}")
        print(f"Всего записей: {len(df)}")
        
        # Сводная статистика
        print("\nСводная статистика:")
        print(f"Всего видео обработано: {len(video_files)}")
        print(f"Всего обнаружений: {len(df)}")
        
        # Статистика по типам мишеней
        target_stats = df.groupby(['target_type', 'target_state']).size().reset_index(name='count')
        print("\nСтатистика по типам мишеней:")
        for _, row in target_stats.iterrows():
            print(f"  {row['target_type']}_{row['target_state']}: {row['count']}")
        
        # Статистика по положениям
        position_stats = df.groupby('target_position').size().reset_index(name='count')
        print("\nСтатистика по положениям мишеней:")
        for _, row in position_stats.iterrows():
            print(f"  {row['target_position']}: {row['count']}")
    
    else:
        print("Мишени не обнаружены ни в одном видео!")

if __name__ == "__main__":
    # Конфигурация
    VIDEO_DIR = "/kaggle/input/mlbdlw4-shots/03-14-2024" 
    MODEL_PATH = "/kaggle/working/best_model.pt" 
    OUTPUT_CSV = "/kaggle/working/video_analysis_results.csv"
    
    # Проверяем доступность GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используемое устройство: {device}")
    
    # Запускаем анализ
    analyze_videos_in_directory(
        video_dir=VIDEO_DIR,
        model_path=MODEL_PATH,
        output_csv=OUTPUT_CSV
    )
    

Используемое устройство: cuda
Загрузка модели...
Найдено видеофайлов: 6

Обработка видео: SHOT0001.MP4
  FPS: 39.96, Кадров: 544, Разрешение: 2624x1472
  Прогресс: 36.6% | Скорость: 60.4 кадр/сек
  Прогресс: 73.2% | Скорость: 59.6 кадр/сек
  Обработано кадров: 544
  Обнаружено мишеней: 182
  Распределение мишеней: {'red_entire': 166, 'dark_entire': 16}

Обработка видео: SHOT0004.MP4
  FPS: 39.96, Кадров: 544, Разрешение: 2624x1472
  Прогресс: 36.6% | Скорость: 63.0 кадр/сек
  Прогресс: 73.2% | Скорость: 62.8 кадр/сек
  Обработано кадров: 544
  Обнаружено мишеней: 321
  Распределение мишеней: {'dark_entire': 97, 'red_entire': 220, 'dark_broken': 4}

Обработка видео: SHOT0005.MP4
  FPS: 39.96, Кадров: 536, Разрешение: 2624x1472
  Прогресс: 37.1% | Скорость: 52.6 кадр/сек
  Прогресс: 74.3% | Скорость: 52.4 кадр/сек
  Обработано кадров: 536
  Обнаружено мишеней: 297
  Распределение мишеней: {'dark_entire': 81, 'red_broken': 4, 'red_entire': 209, 'dark_broken': 3}

Обработка видео: SHOT0003

In [13]:
!pip install pyspark

In [22]:
from pyspark.sql import SparkSession

def longest_series(csv_path):

    spark = SparkSession.builder.appName("SimpleSeries").getOrCreate()
    
    # Читаем и обрабатываем данные
    df = spark.read.option("header", "true").csv(csv_path)
    
    # Получаем номера видео с попаданиями
    hit_videos = df.filter("target_state = 'broken'") \
                   .selectExpr("regexp_extract(video_file, 'SHOT(\\\\d+)\\\\.MP4', 1) as video_num") \
                   .distinct() \
                   .rdd.map(lambda row: int(row.video_num)) \
                   .collect()
    
    spark.stop()
    
    # Сортируем номера видео
    hit_videos.sort()
    
    # Находим самую длинную последовательную серию
    if not hit_videos:
        return 0, []
    
    max_length = 1
    current_length = 1
    max_start = hit_videos[0]
    current_start = hit_videos[0]
    
    for i in range(1, len(hit_videos)):
        if hit_videos[i] == hit_videos[i-1] + 1:
            current_length += 1
        else:
            if current_length > max_length:
                max_length = current_length
                max_start = current_start
            current_length = 1
            current_start = hit_videos[i]
    
    # Проверяем последнюю серию
    if current_length > max_length:
        max_length = current_length
        max_start = current_start
    
    # Создаем список номеров серии
    series = list(range(max_start, max_start + max_length))
    
    return max_length, series

# Использование
length, series = longest_series("/kaggle/working/video_analysis_results.csv")
print(f"Длина самой длинной серии: {length}")
print(f"Видео в серии: {series}")

Длина самой длинной серии: 4
Видео в серии: [2, 3, 4, 5]
